In [2]:
# Step 1: Import Libraries

import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
from scipy.stats import mode

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:
# Step 2: Load Processed Data

# Get the directory of the current script
SCRIPT_DIR = os.getcwd()

DATA_DIR = os.path.join(SCRIPT_DIR, "..", "data/finalized_files")
MODEL_DIR = os.path.join(SCRIPT_DIR, "..", "models")

df_hourly = pd.read_csv(os.path.join(DATA_DIR, "final_hourly_data.csv"), sep=",", parse_dates=["Start date"], low_memory=False)
df_daily = pd.read_csv(os.path.join(DATA_DIR, "final_daily_data.csv"), sep=",", parse_dates=["Start date"], low_memory=False)
df_weekly = pd.read_csv(os.path.join(DATA_DIR, "final_weekly_data.csv"), sep=",", parse_dates=["Start date"], low_memory=False)

# Ensure proper datetime index
df_hourly.set_index("Start date", inplace=True)
df_daily.set_index("Start date", inplace=True)
df_weekly.set_index("Start date", inplace=True)


In [4]:
# Step 3: Feature Engineering

price_columns = [
    "Germany/Luxembourg [/MWh] Original resolutions",
    "Belgium [/MWh] Original resolutions",
    "France [/MWh] Original resolutions"
]

df_hourly["Average_Price_€/MWh"] = df_hourly[price_columns].mean(axis=1)
df_daily["Average_Price_€/MWh"] = df_daily[price_columns].mean(axis=1)
df_weekly["Average_Price_€/MWh"] = df_weekly[price_columns].mean(axis=1)

# Ensure the column exists
if "Average_Price_€/MWh" not in df_hourly.columns:
    raise KeyError("⚠ 'Average_Price_€/MWh' column not found in df_hourly!")

# Compute price direction labels (+1 = Up, -1 = Down, 0 = Stable)
def compute_direction(price_series, threshold=0.1):
    diff = price_series.diff()
    return np.where(diff > threshold, 1, np.where(diff < -threshold, -1, 0))

df_hourly["Price_Direction"] = compute_direction(df_hourly["Average_Price_€/MWh"]).astype(int)
df_daily["Price_Direction"] = compute_direction(df_daily["Average_Price_€/MWh"]).astype(int)
df_weekly["Price_Direction"] = compute_direction(df_weekly["Average_Price_€/MWh"]).astype(int)




In [5]:
# # Step 4: Prepare Data for Baseline & LSTM Models

# Define target & label
TARGET = "Average_Price_€/MWh"
LABEL = "Price_Direction"

# Ensure datetime index
df_hourly = df_hourly.sort_index()
df_daily = df_daily.sort_index()
df_weekly = df_weekly.sort_index()

# Rolling Mean Features
df_hourly["Rolling_Mean_24h"] = df_hourly["Average_Price_€/MWh"].rolling(window=24).mean()
df_daily["Rolling_Mean_7d"] = df_daily["Average_Price_€/MWh"].rolling(window=7).mean()

# Price Differences
df_hourly["Price_Diff"] = df_hourly["Average_Price_€/MWh"].diff()

# Lag Features
df_hourly["Lag_1h"] = df_hourly["Average_Price_€/MWh"].shift(1)
df_hourly["Lag_24h"] = df_hourly["Average_Price_€/MWh"].shift(24)

# Volatility (Rolling Standard Deviation)
df_hourly["Volatility_24h"] = df_hourly["Average_Price_€/MWh"].rolling(window=24).std()

# Percentage Change in Price
df_hourly["Price_Change_1h"] = df_hourly["Average_Price_€/MWh"].pct_change(1) * 100
df_hourly["Price_Change_24h"] = df_hourly["Average_Price_€/MWh"].pct_change(24) * 100

# Fill NaN values after rolling/shift operations
df_hourly.fillna(0, inplace=True)
df_daily.fillna(0, inplace=True)
df_weekly.fillna(0, inplace=True)

# Expected feature sets per dataset
expected_features_hourly = [
    "Rolling_Mean_24h", "Price_Diff", "Lag_1h", "Lag_24h",
    "Volatility_24h", "Price_Change_1h", "Price_Change_24h"
]
expected_features_daily = ["Rolling_Mean_7d"]

# Ensure weekly features exist
df_weekly["Rolling_Mean_4w"] = df_weekly["Average_Price_€/MWh"].rolling(window=4).mean()
df_weekly.fillna(0, inplace=True)  # Fill NaNs after rolling
expected_features_weekly = ["Rolling_Mean_4w"]

# Feature selection
X_hourly, y_hourly = df_hourly[expected_features_hourly], df_hourly[LABEL]
X_daily, y_daily = df_daily[expected_features_daily], df_daily[LABEL]
X_weekly, y_weekly = df_weekly[expected_features_weekly], df_weekly[LABEL]

# Debugging: Check for issues in feature matrices
for name, X in [("Hourly", X_hourly), ("Daily", X_daily), ("Weekly", X_weekly)]:
    if np.any(np.isnan(X)):
        print(f"⚠ NaN values found in {name} dataset")
    if np.any(np.isinf(X)):
        print(f"⚠ Infinite values found in {name} dataset")
    if np.any(X > 1e6):  # Adjust threshold if needed
        print(f"⚠ Extremely large values detected in {name} dataset")

# Fix infinite values & extremely large numbers
for X in [X_hourly, X_daily, X_weekly]:
    X.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infinities
    X.fillna(0, inplace=True)  # Fill NaNs (use mean/median instead of 0 if needed)
    X.clip(-1e6, 1e6, inplace=True)  # Cap extreme values

# Normalize features for LSTM
scaler = MinMaxScaler()

X_hourly_scaled = scaler.fit_transform(X_hourly) if not X_hourly.empty else np.array([])
X_daily_scaled = scaler.fit_transform(X_daily) if not X_daily.empty else np.array([])
X_weekly_scaled = scaler.fit_transform(X_weekly) if not X_weekly.empty else np.array([])

# Train-Test Split
if X_hourly_scaled.size > 0:
    X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_hourly_scaled, y_hourly, test_size=0.2, random_state=42)
if X_daily_scaled.size > 0:
    X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_daily_scaled, y_daily, test_size=0.2, random_state=42)
if X_weekly_scaled.size > 0:
    X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_weekly_scaled, y_weekly, test_size=0.2, random_state=42)

# Reshape for LSTM (samples, time steps, features)
if X_hourly_scaled.size > 0:
    X_train_h_lstm = X_train_h.reshape((X_train_h.shape[0], 1, X_train_h.shape[1]))
    X_test_h_lstm = X_test_h.reshape((X_test_h.shape[0], 1, X_test_h.shape[1]))

if X_daily_scaled.size > 0:
    X_train_d_lstm = X_train_d.reshape((X_train_d.shape[0], 1, X_train_d.shape[1]))
    X_test_d_lstm = X_test_d.reshape((X_test_d.shape[0], 1, X_test_d.shape[1]))

if X_weekly_scaled.size > 0:
    X_train_w_lstm = X_train_w.reshape((X_train_w.shape[0], 1, X_train_w.shape[1]))
    X_test_w_lstm = X_test_w.reshape((X_test_w.shape[0], 1, X_test_w.shape[1]))

print("✅ Data preparation completed successfully!")




✅ Data preparation completed successfully!


C:\Users\Elite\AppData\Local\Temp\ipykernel_30732\2753602932.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_hourly["Price_Change_1h"] = df_hourly["Average_Price_€/MWh"].pct_change(1) * 100
C:\Users\Elite\AppData\Local\Temp\ipykernel_30732\2753602932.py:28: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_hourly["Price_Change_24h"] = df_hourly["Average_Price_€/MWh"].pct_change(24) * 100
C:\Users\Elite\AppData\Local\Temp\ipykernel_30732\2753602932.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:

In [6]:
# # # 🛠 Step 5: Ensure `y_train_h` contains multiple classes

# 🛠 Step 1: Fix `Price_Direction` Computation
def compute_direction(prices, threshold=0.05):
    """
    Computes price direction based on percentage change.
    Returns: 1 (rise), -1 (fall), 0 (stable)
    """
    price_change = prices.pct_change().fillna(0)
    direction = np.where(price_change > threshold, 1, np.where(price_change < -threshold, -1, 0))
    return direction

df_hourly["Price_Direction"] = compute_direction(df_hourly["Average_Price_€/MWh"], threshold=0.05)
df_hourly["Price_Direction"] = df_hourly["Price_Direction"].fillna(0).astype(int)

# Target variable
y = df_hourly["Price_Direction"]

# Features (Drop unnecessary columns)
X = df_hourly.drop(columns=["Timestamp", "Price_Direction"], errors="ignore")

# 🛠 Step 2: Handle NaN/Infinite Values in `X`
X.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infinities with NaN
X.fillna(0, inplace=True)  # Fill NaNs with 0

# 🏗 Step 3: Split Train/Test Data
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 🔄 Adjust Labels to Start from 0
y_train_h = y_train_h.replace({-1: 0, 0: 1, 1: 2})
y_test_h = y_test_h.replace({-1: 0, 0: 1, 1: 2})

# 🛠 Step 4: Handle Class Imbalance (SMOTE)
if len(np.unique(y_train_h)) == 1:
    print("⚠ Warning: Only one class detected! Applying SMOTE to balance classes.")
    smote = SMOTE(random_state=42)
    X_train_h, y_train_h = smote.fit_resample(X_train_h, y_train_h)

# print("Class distribution after SMOTE:\n", pd.Series(y_train_h).value_counts())

# 🛠 Step 5: Ensure Consistency Between Train & Test Features
X_test_h = X_test_h[X_train_h.columns]  # Keep only features present in training

# 🏆 Step 6: Train Baseline Models (LightGBM & XGBoost)
lgb_model = lgb.LGBMClassifier()
xgb_model = xgb.XGBClassifier()

# 🚀 Fix Feature Names by Removing Special Characters
X_train_h.columns = [col.replace(" ", "_").replace("{", "").replace("}", "").replace("[", "").replace("]", "").replace(":", "").replace("\\", "").replace("\"", "") for col in X_train_h.columns]
X_test_h.columns = X_train_h.columns  # Ensure test set has the same feature names

# 🔎 Check the first few column names after fixing
print("🛠 Fixed Feature Names:", X_train_h.columns.tolist()[:10])

lgb_model.fit(X_train_h, y_train_h)
xgb_model.fit(X_train_h, y_train_h)

# Predictions
y_pred_lgb = lgb_model.predict(X_test_h)
y_pred_xgb = xgb_model.predict(X_test_h)

# Evaluation
print("✅ LightGBM Accuracy:", accuracy_score(y_test_h, y_pred_lgb))
print("✅ XGBoost Accuracy:", accuracy_score(y_test_h, y_pred_xgb))




🛠 Fixed Feature Names: ['End_date', 'Germany/Luxembourg_/MWh_Original_resolutions', '_DE/LU_neighbours_/MWh_Original_resolutions', 'Belgium_/MWh_Original_resolutions', 'Denmark_1_/MWh_Original_resolutions', 'Denmark_2_/MWh_Original_resolutions', 'France_/MWh_Original_resolutions', 'Netherlands_/MWh_Original_resolutions', 'Norway_2_/MWh_Original_resolutions', 'Austria_/MWh_Original_resolutions']
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019351 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22505
[LightGBM] [Info] Number of data points in the train set: 15244, number of used features: 91
[LightGBM] [Info] Start training from score -2.349868
[LightGBM] [Info] Start training from score -0.255663
[LightGBM] [Info] Start training from score -2.038567
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [8]:
# Step 6: Ensemble Predictions

# Majority voting from LGBM, XGBoost, and LSTM
ensemble_preds = np.round((y_pred_lgb + y_pred_xgb) / 3)

# Final Accuracy
print("Ensemble Model Accuracy:", accuracy_score(y_test_h, ensemble_preds))

# ⚠️ Volatility Capture 
actual_volatility = y_test_h.diff().std()
predicted_volatility = pd.Series(ensemble_preds).diff().std()
print(f"📈 Actual Volatility: {actual_volatility:.4f}")
print(f"🔮 Predicted Volatility: {predicted_volatility:.4f}")

# ⚠️ Extreme Price Movement Detection
y_test_h = pd.Series(y_test_h)
extreme_moves = (y_test_h.abs() > 0.15).sum()
extreme_correct = ((y_test_h.abs() > 0.15) & (np.abs(ensemble_preds) > 0.15)).sum()
extreme_accuracy = extreme_correct / extreme_moves if extreme_moves > 0 else 0
print(f"📊 Extreme Price Movement Accuracy: {extreme_accuracy:.4f}")



Ensemble Model Accuracy: 0.8688352570828961
📈 Actual Volatility: 0.6577
🔮 Predicted Volatility: 0.4124
📊 Extreme Price Movement Accuracy: 0.9985


In [9]:
# Step 7: Evaluate the Model

def evaluate_model(y_true, y_pred):
    # Convert to Pandas Series if needed
    y_true = pd.Series(y_true)
    y_pred = pd.Series(y_pred)

    # ✅ Directional Accuracy
    directional_acc = accuracy_score(y_true, y_pred)

    # 📉 Mean Absolute Error (MAE) & RMSE
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # 📊 Volatility Capture (Compare Standard Deviations)
    true_volatility = np.std(y_true)
    pred_volatility = np.std(y_pred)
    volatility_capture = 1 - abs(true_volatility - pred_volatility) / true_volatility if true_volatility != 0 else 0

    # ⚠️ Extreme Price Movement Accuracy
    extreme_moves = (y_test_h.abs() > 0.15).sum()
    extreme_correct = ((y_true.abs() > 0.15) & (np.abs(ensemble_preds) > 0.15)).sum()
    extreme_accuracy = extreme_correct / extreme_moves if extreme_moves > 0 else 0

    # 🔥 Print Evaluation Metrics
    print(f"✅ Directional Accuracy: {directional_acc:.4f}")
    print(f"📉 Mean Absolute Error (MAE): {mae:.4f}")
    print(f"📊 Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"📊 Volatility Capture Score: {volatility_capture:.4f}")
    print(f"⚠️ Extreme Price Movement Accuracy: {extreme_accuracy:.4f}")

# 🔎 Run Evaluation with New Metrics
evaluate_model(y_test_h, ensemble_preds)



✅ Directional Accuracy: 0.8688
📉 Mean Absolute Error (MAE): 0.1314
📊 Root Mean Squared Error (RMSE): 0.3633
📊 Volatility Capture Score: 0.6242
⚠️ Extreme Price Movement Accuracy: 0.9985


In [11]:
# Step 8: Save the Model

# Save models
joblib.dump(lgb_model, os.path.join(MODEL_DIR, "lgb_market_model.pkl"))
joblib.dump(xgb_model, os.path.join(MODEL_DIR, "xgb_market_model.pkl"))

print("✅ Models saved successfully!")


✅ Models saved successfully!
